In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import concurrent.futures
import re
import statsapi
import pprint
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)



In [2]:
# sched = statsapi.schedule(start_date='07/01/2010',end_date='07/31/2018',team=143)
#statsapi.get('team', {'teamId':143})


def findTeamKeys():
    mlbTeamKeys = []
    url = "https://statsapi.mlb.com/api/v1/teams/"
    response = requests.get(url)



    data = response.json()
    for team in data['teams']:
        #if team.get('sport', {}).get('name', '') == "Major League Baseball":
        mlbTeamKeys.append(
            {
            'Team' : team.get('name'),
            'team_id' : team.get('id'),
            'team_abbr' : team.get('abbreviation'),
            'league' : team.get('league',{}).get('name'),
            'division' : team.get('division',{}).get('name'),
            'sport' : team.get('sport', {}).get('name')
            }
        )
    return pd.DataFrame(mlbTeamKeys)

statsApiTeamKeys = findTeamKeys()
mlbTeamKeys = statsApiTeamKeys[statsApiTeamKeys['sport'] == 'Major League Baseball']


# Define the start and end dates
start_date = datetime.strptime('03/10/2015', '%m/%d/%Y')
end_date = datetime.now()

# Initialize the list to hold the date ranges
date_ranges = []

# Loop to generate the dates in 30 day increments
current_date = start_date
while current_date <= end_date:
    # Add the current date to the list
    date_ranges.append(current_date.strftime('%m/%d/%Y'))
    # Increment the current date by 30 days
    current_date += timedelta(days=30)
date_ranges.append(end_date.strftime('%m/%d/%Y'))

# Function to fetch schedules for a given team ID and date range
def fetch_schedule_for_team_date_range(team_id, start_date, end_date):
    sched = statsapi.schedule(start_date=start_date, end_date=end_date, team=team_id)
    return sched

# Main function to fetch schedules using multithreading
def fetch_schedules_multithreaded(date_ranges, team_ids):
    fullSched = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
        # Prepare future tasks for each combination of team ID and date range
        future_to_sched = {
            executor.submit(fetch_schedule_for_team_date_range, team_id, date_ranges[index-1], date_ranges[index]): (team_id, index)
            for index in range(1, len(date_ranges))
            for team_id in team_ids
        }
        # As each future completes, append its result to fullSched
        for future in concurrent.futures.as_completed(future_to_sched):
            sched = future.result()
            if sched:
                for row in sched:
                    fullSched.append(row)
    return fullSched

# Example usage
team_ids = mlbTeamKeys['team_id'].to_list()  # Assuming this list is available
fullSched = fetch_schedules_multithreaded(date_ranges, team_ids)
fullSched = pd.DataFrame(fullSched)


In [ ]:
fullSched = fullSched.drop_duplicates(subset=['game_id'], keep='first')

In [4]:
fullSched.to_csv('basicGameLogs.csv', index=False)

In [2]:
df = pd.read_csv('basicGameLogs.csv')


In [3]:
set(df['game_type'])
df = df.sort_values(by='game_date')
df = df[df['status'] != 'Cancelled']
df = df[df['game_type']=='R']
df = df[df['status'] == 'Final']

In [4]:
df

,game_id,game_datetime,game_date,game_type,status,away_name,home_name,away_id,home_id,doubleheader,game_num,home_probable_pitcher,away_probable_pitcher,home_pitcher_note,away_pitcher_note,away_score,home_score,current_inning,inning_state,venue_id,venue_name,national_broadcasts,series_status,winning_team,losing_team,winning_pitcher,losing_pitcher,save_pitcher,summary,losing_Team
361,413661,2015-04-06T00:05:00Z,2015-04-05,R,Final,St. Louis Cardinals,Chicago Cubs,138,112,N,1,Jon Lester,Adam Wainwright,Lester will make his second career start at Wr...,Wainwright draws the fifth Opening Day assignm...,3,0,9.0,Bottom,17,Wrigley Field,['ESPN2'],STL leads 1-0,St. Louis Cardinals,Chicago Cubs,Adam Wainwright,Jon Lester,Trevor Rosenthal,2015-04-05 - St. Louis Cardinals (3) @ Chicago...,NaN
375,413663,2015-04-06T17:05:00Z,2015-04-06,R,Final,Toronto Blue Jays,New York Yankees,141,147,N,1,Masahiro Tanaka,Drew Hutchison,Tanaka draws his first career Opening Day assi...,Hutchison will make his first Opening Day star...,6,1,9.0,Bottom,3313,Yankee Stadium,['ESPN'],TOR leads 1-0,Toronto Blue Jays,New York Yankees,Drew Hutchison,Masahiro Tanaka,NaN,2015-04-06 - Toronto Blue Jays (6) @ New York ...,NaN
374,413660,2015-04-07T02:10:00Z,2015-04-06,R,Final,San Francisco Giants,Arizona Diamondbacks,137,109,N,1,Josh Collmenter,Madison Bumgarner,Collmenter elevates to his first Opening Day s...,Bumgarner began 2014 the way he's starting thi...,5,4,9.0,Bottom,15,Chase Field,['ESPN2'],SF leads 1-0,San Francisco Giants,Arizona Diamondbacks,Madison Bumgarner,Josh Collmenter,Santiago Casilla,2015-04-06 - San Francisco Giants (5) @ Arizon...,NaN
373,413650,2015-04-06T20:10:00Z,2015-04-06,R,Final,Atlanta Braves,Miami Marlins,144,146,N,1,Henderson Alvarez III,Julio Teheran,An All-Star last year with a no-hitter under h...,Teheran will enter his third full season in At...,2,1,9.0,Bottom,4169,Marlins Park,[],ATL leads 1-0,Atlanta Braves,Miami Marlins,Julio Teheran,Henderson Alvarez III,Jason Grilli,2015-04-06 - Atlanta Braves (2) @ Miami Marlin...,NaN
372,413662,2015-04-07T02:05:00Z,2015-04-06,R,Final,Texas Rangers,Oakland Athletics,140,133,N,1,Sonny Gray,Yovani Gallardo,"Gray, 25, will be making his second straight O...",Gallardo is making his sixth straight Opening ...,0,8,9.0,Top,10,O.co Coliseum,[],OAK leads 1-0,Oakland Athletics,Texas Rangers,Sonny Gray,Yovani Gallardo,NaN,2015-04-06 - Texas Rangers (0) @ Oakland Athle...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24311,745523,2024-04-07T17:35:00Z,2024-04-07,R,Final,Baltimore Orioles,Pittsburgh Pirates,110,134,N,1,Marco Gonzales,Dean Kremer,NaN,NaN,2,3,9.0,Bottom,31,PNC Park,[],PIT wins 2-1,Pittsburgh Pirates,Baltimore Orioles,Jose Hernandez,Yennier Cano,NaN,2024-04-07 - Baltimore Orioles (2) @ Pittsburg...,NaN
24310,745362,2024-04-07T20:05:00Z,2024-04-07,R,Final,San Diego Padres,San Francisco Giants,135,137,N,1,Logan Webb,Matt Waldron,NaN,NaN,2,3,9.0,Top,2395,Oracle Park,[],SF wins 2-1,San Francisco Giants,San Diego Padres,Ryan Walker,Jhony Brito,Camilo Doval,2024-04-07 - San Diego Padres (2) @ San Franci...,NaN
24320,746574,2024-04-07T19:10:00Z,2024-04-07,R,Final,Tampa Bay Rays,Colorado Rockies,139,115,N,1,Dakota Hudson,Ryan Pepiot,NaN,NaN,3,2,9.0,Bottom,19,Coors Field,[],TB wins 2-1,Tampa Bay Rays,Colorado Rockies,Ryan Pepiot,Dakota Hudson,Shawn Armstrong,2024-04-07 - Tampa Bay Rays (3) @ Colorado Roc...,NaN
24314,746330,2024-04-07T18:10:00Z,2024-04-07,R,Final,Chicago White Sox,Kansas City Royals,145,118,N,1,Alec Marsh,Garrett Crochet,NaN,NaN,3,5,9.0,Top,7,Kauffman Stadium,[],KC wins 4-0,Kansas City Royals,Chicago White Sox,John Schreiber,Deivi García,James McArthur,2024-04-07 - Chicago White Sox (3) @ Kansas Ci...,NaN


In [5]:
game_id_list = df['game_id'].to_list()
game_id_list = list(set(game_id_list))

In [6]:
len(df)

20008

In [351]:
import pandas as pd
import statsapi

def fetch_and_process_game_data(game_id, game_date):
    max_attempts = 3
    attempt = 0
    while attempt< max_attempts:
        try:
            # Fetch the boxscore data
            boxscore = statsapi.boxscore_data(game_id, timecode=None)

            # Initialize lists to store data
            batter_boxscore = []
            pitcher_boxscore = []

            # Extract team information
            home_team = f"{boxscore['teamInfo']['home']['shortName']} {boxscore['teamInfo']['home']['teamName']}"
            away_team = f"{boxscore['teamInfo']['away']['shortName']} {boxscore['teamInfo']['away']['teamName']}"
            home_id = boxscore['teamInfo']['home']['id']
            away_id = boxscore['teamInfo']['away']['id']

            # Process batters and pitchers for both teams
            for batter in boxscore['awayBatters'][1:] + boxscore['homeBatters'][1:]:
                batter_boxscore.append({
                    'Team': away_team if batter in boxscore['awayBatters'][1:] else home_team,
                    'team_id': away_id if batter in boxscore['awayBatters'][1:] else home_id,
                    **batter
                })

            for pitcher in boxscore['awayPitchers'][1:] + boxscore['homePitchers'][1:]:
                pitcher_boxscore.append({
                    'Team': away_team if pitcher in boxscore['awayPitchers'][1:] else home_team,
                    'team_id': away_id if pitcher in boxscore['awayPitchers'][1:] else home_id,
                    **pitcher
                })

            # Convert lists to DataFrames
            batter_boxscore_df = pd.DataFrame(batter_boxscore)
            pitcher_boxscore_df = pd.DataFrame(pitcher_boxscore)

            # Add game_id and game_date to both DataFrames
            for df in [batter_boxscore_df, pitcher_boxscore_df]:
                df['game_id'] = game_id
                df['date'] = game_date

                # Calculate 'TB' for batters
                if 'doubles' in df.columns and 'triples' in df.columns and 'hr' in df.columns and 'h' in df.columns:
                    df['TB'] = (df['doubles'].astype(int) * 2 + 
                                df['triples'].astype(int) * 3 + 
                                df['hr'].astype(int) * 4 + 
                                (df['h'].astype(int) - df['doubles'].astype(int) - df['triples'].astype(int) - df['hr'].astype(int)))

                # Example merging or additional processing could go here
            id_df = []
            for player_id in boxscore['playerInfo'].keys():
                id_df.append( {
                    'personId' : boxscore['playerInfo'][player_id]['id'],
                    'Name' : boxscore['playerInfo'][player_id]['fullName']
                })

            id_df = pd.DataFrame(id_df)

            batter_boxscore_df = batter_boxscore_df.merge(id_df, how='left', on='personId')
            pitcher_boxscore_df = pitcher_boxscore_df.merge(id_df, how='left', on='personId')
                
            # Return processed DataFrames
            return batter_boxscore_df, pitcher_boxscore_df
        except Exception as e:
            print(f'Error processing game ID {game_id}: {e} on attempt {attempt}')
            time.sleep(2**attempt)
            attempt += 1
            # Return empty DataFrames in case of an error to maintain consistency
    if attempt == max_attempts:
        print(f"Failed to fetch data after maximum attempts for {game_id}")
        return pd.DataFrame(), pd.DataFrame()





def process_all_games(df):
    all_batter_boxscores = pd.DataFrame()
    all_pitcher_boxscores = pd.DataFrame()

    with ThreadPoolExecutor(max_workers=200) as executor:
        futures = [executor.submit(fetch_and_process_game_data, row['game_id'], row['game_date']) for index, row in df.iterrows()]

        for future in as_completed(futures):
            try:
                batter_boxscore, pitcher_boxscore = future.result()
                all_batter_boxscores = pd.concat([all_batter_boxscores, batter_boxscore], ignore_index=True)
                all_pitcher_boxscores = pd.concat([all_pitcher_boxscores, pitcher_boxscore], ignore_index=True)
            except Exception as e:
                # Handle exception, perhaps logging the game_id that caused it
                print(f'An error occurred: {e}')

    return all_batter_boxscores, all_pitcher_boxscores


In [352]:
# Assuming 'df' is your DataFrame with game_ids and game_dates
# and 'df_template' is the DataFrame structure used for merging operations
all_batter_boxscores, all_pitcher_boxscores = process_all_games(df)

#all_batter_boxscores = all_batter_boxscores.merge(id_df, how='left', on='personId')
all_batter_boxscores = all_batter_boxscores.merge(df, how='left', on='game_id')

all_pitcher_boxscores = all_pitcher_boxscores.merge(df,how='left', on='game_id')
#all_pitcher_boxscores = all_pitcher_boxscores.merge(id_df, how='left', on='personId')

all_batter_boxscores['isWinner'] = all_batter_boxscores['Team'] == all_batter_boxscores['winning_team']
all_pitcher_boxscores['isWinner'] = all_pitcher_boxscores['Team'] == all_pitcher_boxscores['winning_team']
all_batter_boxscores['isStarter'] = (all_batter_boxscores['battingOrder'].astype(int) % 100 == 0)

In [355]:
all_batter_boxscores.to_csv('batterBoxscores.csv', index=False)
all_pitcher_boxscores.to_csv('pitcherBoxscores.csv', index=False)


In [6]:
all_strikeouts = pd.DataFrame([])
all_walks = pd.DataFrame([])
all_hits_allowed = pd.DataFrame([])
all_play_by_play = pd.DataFrame([])

In [7]:
game_id_list
temp = game_id_list[:500]
len(game_id_list)

20008

In [8]:


#for game_id in temp:
def grabMatrices(game_id):
    max_attempts = 5
    attempt = 0
    while attempt< max_attempts:
        try:
            def getPlayByPlay(game_id):
                
                play_by_play = statsapi.get('game_playByPlay', {'gamePk' : game_id})
                all_plays = play_by_play['allPlays']

                game_play_by_play_df = []
                simple_game_play_by_play_df = []
                for play in all_plays:
                    batter_id = play.get('matchup',{}).get('batter',{}).get('id')
                    pitcher_id = play.get('matchup',{}).get('pitcher',{}).get('id')
                    batter_name = play.get('matchup',{}).get('batter',{}).get('fullName')
                    pitcher_name = play.get('matchup',{}).get('pitcher',{}).get('fullName')

                    simple_game_play_by_play_df.append(
                        {
                            'game_ID' : game_id,
                            'atBatResult' : play.get('result',{}).get('event'),
                            'halfInning' : play.get('about', {}).get('halfInning'),
                            'inning' : play.get('about',{}).get('inning'),
                            'batter_id' : batter_id,
                            'pitcher_id' : pitcher_id,
                            'batter_name' : batter_name,
                            'pitcher_name' : pitcher_name,
                            'batter_hand' : play.get('matchup',{}).get('batSide',{}).get('code'),
                            'pitcher_hand' : play.get('matchup',{}).get('pitchHand',{}).get('code')
                            
                        }
                    )

                    for pitch in play['playEvents']:

                        if pitch['isPitch']:

                            pitch_dict = {
                                'game_ID' : game_id,
                                'atBatResult' : play.get('result',{}).get('event'),
                                'halfInning' : play.get('about', {}).get('halfInning'),
                                'inning' : play.get('about',{}).get('inning'),
                                'batter_id' : batter_id,
                                'pitcher_id' : pitcher_id,
                                'batter_name' : batter_name,
                                'pitcher_name' : pitcher_name,
                                'batter_hand' : play.get('matchup',{}).get('batSide',{}).get('code'),
                                'pitcher_hand' : play.get('matchup',{}).get('pitchHand',{}).get('code'),
                                'play_description' : pitch.get('details', {}).get('description'),
                                'isInPlay' : pitch.get('details', {}).get('isInPlay'),
                                'isStrike' : pitch.get('details', {}).get('isStrike'),
                                'isBall' : pitch.get('details', {}).get('isBall'),
                                'pitchNumber' : pitch.get('pitchNumber'),
                                'pitch_type' : pitch.get('details', {}).get('type',{}).get('description'),
                                'isOut' : pitch.get('details', {}).get('isOut'),
                                'current_balls' : pitch.get('count',{}).get('balls'),
                                'current_strikes' : pitch.get('count', {}).get('strikes'),
                                'current_outs' : pitch.get('count', {}).get('outs'),
                                'pitchStartSpeed' : pitch.get('pitchData',{}).get('startSpeed'),
                                'pitchEndSpeed' : pitch.get('pitchData', {}).get('endSpeed'),
                                'pitchStrikeZoneTop' : pitch.get('pitchData', {}).get('strikeZoneTop'),
                                'pitchStikeZoneBottom' : pitch.get('pitchData', {}).get('strikeZoneBottom'),
                                'pitchZone' : pitch.get('pitchData',{}).get('zone'),
                                'pitchTypeConfidence' : pitch.get('pitchData', {}).get('typeConfidence')

                            }

                            for coord in pitch.get('pitchData',{}).get('coordinates',{}).keys():
                                pitch_dict[coord] = pitch.get('pitchData',{}).get('coordinates',{}).get(coord)
                            for breaks in pitch.get('pitchData',{}).get('breaks',{}).keys():
                                pitch_dict[breaks] = pitch.get('pitchData',{}).get('breaks',{}).get(breaks)

                            game_play_by_play_df.append(pitch_dict)    


                play_by_play = pd.DataFrame(game_play_by_play_df)
                simple_game_play_by_play_df = pd.DataFrame(simple_game_play_by_play_df)

                play_by_play['pitchesThrownByCurrentPitcher'] = play_by_play.groupby('pitcher_id').cumcount() + 1
                # Initialize columns to zero in the play_by_play DataFrame
                play_by_play['cumulativeBalls'] = 0
                play_by_play['cumulativeStrikes'] = 0

                # Find indices of balls and strikes in the play_by_play DataFrame
                ball_indices = play_by_play[play_by_play['isBall'] == True].index
                strike_indices = play_by_play[play_by_play['isStrike'] == True].index

                # Update columns with correct cumulative counts in the play_by_play DataFrame
                play_by_play.loc[ball_indices, 'cumulativeBalls'] = play_by_play.loc[ball_indices].groupby('pitcher_id').cumcount() + 1
                play_by_play.loc[strike_indices, 'cumulativeStrikes'] = play_by_play.loc[strike_indices].groupby('pitcher_id').cumcount() + 1

                # Define a function to count strikeouts and hits allowed
                def count_stats(row):
                    if row['atBatResult'] == 'Strikeout':
                        return 'Strikeout'
                    elif row['atBatResult'] in ['Single', 'Double', 'Triple', 'Home Run']:
                        return 'Hit'
                    else:
                        return 'Other'

                # Apply the function to categorize each pitch
                play_by_play['stat_category'] = play_by_play.apply(count_stats, axis=1)

                return play_by_play, simple_game_play_by_play_df

            play_by_play, simple_game_play_by_play_df = getPlayByPlay(game_id)

            # Initialize a dictionary to hold our DataFrames
            stats_matrices = {}

            # Pitch Counts (already created)
            #stats_matrices['Pitch Counts'] = pitcher_inning_matrix

            # Strikeouts per Inning
            strikeouts_per_inning = simple_game_play_by_play_df[simple_game_play_by_play_df['atBatResult'] == 'Strikeout'].groupby(['pitcher_name', 'inning']).size().reset_index(name='strikeouts')
            strikeouts_matrix = strikeouts_per_inning.pivot(index='pitcher_name', columns='inning', values='strikeouts').fillna(0).astype(int)
            stats_matrices['Strikeouts'] = strikeouts_matrix

            # Hits Allowed per Inning
            hits_allowed_per_inning = simple_game_play_by_play_df[simple_game_play_by_play_df['atBatResult'].isin(['Single', 'Double', 'Triple', 'Home Run'])].groupby(['pitcher_name', 'inning']).size().reset_index(name='hits_allowed')
            hits_matrix = hits_allowed_per_inning.pivot(index='pitcher_name', columns='inning', values='hits_allowed').fillna(0).astype(int)
            stats_matrices['Hits Allowed'] = hits_matrix

            # walks per Inning
            walks_per_inning = simple_game_play_by_play_df[simple_game_play_by_play_df['atBatResult'] == 'Walk'].groupby(['pitcher_name', 'inning']).size().reset_index(name='walks')
            walks_matrix = walks_per_inning.pivot(index='pitcher_name', columns='inning', values='walks').fillna(0).astype(int)
            stats_matrices['Walks'] = walks_matrix


            # Add a 'Total' column to each DataFrame in the dictionary
            for stat, df in stats_matrices.items():
                df['Total'] = df.sum(axis=1)
                df['game_id'] = game_id



            # Define all possible innings based on the play_by_play DataFrame to ensure we cover all innings in the game
            all_innings_sorted = sorted(play_by_play['inning'].unique())

            def ensure_all_innings_and_sort(df, all_innings_sorted):
                # Add missing inning columns with zeros
                for inning in all_innings_sorted:
                    if inning not in df.columns:
                        df[inning] = 0
                # Ensure the innings are in the correct order and append the 'Total' column last
                final_columns = [i for i in range(1, 10)] + ['Total', 'game_id']
                # ordered_columns = sorted(df.columns, key=lambda x: x if isinstance(x, int) else float('inf'))
                # if 'Total' in ordered_columns:
                #     ordered_columns.remove('Total')
                # ordered_columns += ['Total']  # Make sure 'Total' is always the last column
                return df[final_columns]
                

            # This function can be applied to any of the statistical DataFrames like so:
            stats_matrices['Strikeouts'] = ensure_all_innings_and_sort(stats_matrices['Strikeouts'], all_innings_sorted)
            stats_matrices['Hits Allowed'] = ensure_all_innings_and_sort(stats_matrices['Hits Allowed'], all_innings_sorted)
            stats_matrices['Walks'] = ensure_all_innings_and_sort(stats_matrices['Walks'], all_innings_sorted)

            return play_by_play, stats_matrices['Strikeouts'], stats_matrices['Walks'], stats_matrices['Hits Allowed']
        except Exception as e:
            print(f'Error processing game ID {game_id}: {e} on attempt {attempt}')
            time.sleep(2**attempt)
            attempt += 1
    if attempt == max_attempts:
        print(f"Failed to fetch data after maximum attempts for {game_id}")
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
        



def process_games(game_ids):
    all_strikeouts = pd.DataFrame([])
    all_walks = pd.DataFrame([])
    all_hits_allowed = pd.DataFrame([])
    all_play_by_play = pd.DataFrame([])

    with ThreadPoolExecutor(max_workers=250) as executor:
        futures = [executor.submit(grabMatrices, game_id) for game_id in game_ids]
        for future in futures:
            play_by_play_df, strikeouts_df, walks_df, hits_allowed_df = future.result()
            all_strikeouts = pd.concat([all_strikeouts, strikeouts_df])
            all_walks = pd.concat([all_walks, walks_df])
            all_hits_allowed = pd.concat([all_hits_allowed, hits_allowed_df])
            all_play_by_play = pd.concat([all_play_by_play, play_by_play_df])
    
    return all_strikeouts, all_walks, all_hits_allowed, all_play_by_play


In [36]:
temp = game_id_list[18000:]
strikeouts_matrix, walks_matrix, hits_matrix, play_by_play = process_games(temp)
all_play_by_play = pd.concat([all_play_by_play, play_by_play])
all_strikeouts = pd.concat([all_strikeouts, strikeouts_matrix])
all_walks = pd.concat([all_walks, walks_matrix])
all_hits_allowed = pd.concat([all_hits_allowed, hits_matrix])


Error processing game ID 632611: '[8, 9] not in index' on attempt 0
Error processing game ID 632679: '[8, 9] not in index' on attempt 0
Error processing game ID 632738: '[8, 9] not in index' on attempt 0
Error processing game ID 632731: '[8, 9] not in index' on attempt 0
Error processing game ID 632611: '[8, 9] not in index' on attempt 1
Error processing game ID 632679: '[8, 9] not in index' on attempt 1
Error processing game ID 632897: '[8, 9] not in index' on attempt 0
Error processing game ID 632893: '[8, 9] not in index' on attempt 0
Error processing game ID 632738: '[8, 9] not in index' on attempt 1
Error processing game ID 632731: '[8, 9] not in index' on attempt 1
Error processing game ID 632940: '[8, 9] not in index' on attempt 0
Error processing game ID 632949: '[8, 9] not in index' on attempt 0
Error processing game ID 632963: '[8, 9] not in index' on attempt 0
Error processing game ID 633000: '[8, 9] not in index' on attempt 0
Error processing game ID 633081: '[8, 9] not in 

/var/folders/7f/nl87050s61973ky11lnw6gyr0000gn/T/ipykernel_52918/3436725565.py:186: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_play_by_play = pd.concat([all_play_by_play, play_by_play_df])


Error processing game ID 634477: '[8, 9] not in index' on attempt 2
Error processing game ID 634330: '[8, 9] not in index' on attempt 2
Error processing game ID 634166: '[8, 9] not in index' on attempt 3
Error processing game ID 634250: '[8, 9] not in index' on attempt 3
Error processing game ID 633499: '[8, 9] not in index' on attempt 4
Error processing game ID 634532: '[8, 9] not in index' on attempt 2
Error processing game ID 633545: '[9] not in index' on attempt 4
Error processing game ID 634246: '[8, 9] not in index' on attempt 3
Error processing game ID 634574: '[8, 9] not in index' on attempt 2
Error processing game ID 634527: '[8, 9] not in index' on attempt 2
Error processing game ID 634571: '[8, 9] not in index' on attempt 2
Error processing game ID 633432: '[8, 9] not in index' on attempt 4
Failed to fetch data after maximum attempts for 633145
Failed to fetch data after maximum attempts for 633295
Error processing game ID 634208: '[8, 9] not in index' on attempt 3
Error pro

In [37]:
missing_game = [661565]

len(all_play_by_play['game_ID'].unique())


19894

In [42]:
all_strikeouts.to_csv('k_per_inning.csv', index=False)
all_walks.to_csv('bb_per_inning.csv', index=False)
all_hits_allowed.to_csv('h_per_inning.csv', index=False)
all_play_by_play.to_csv('all_play_by_play.csv', index=False)

In [36]:
df[df['game_id'] == 631136]

,game_id,game_datetime,game_date,game_type,status,away_name,home_name,away_id,home_id,doubleheader,game_num,home_probable_pitcher,away_probable_pitcher,home_pitcher_note,away_pitcher_note,away_score,home_score,current_inning,inning_state,venue_id,venue_name,national_broadcasts,series_status,winning_team,losing_team,winning_pitcher,losing_pitcher,save_pitcher,summary,losing_Team
14806,631136,2020-08-09T17:05:00Z,2020-08-09,R,Final,Atlanta Braves,Philadelphia Phillies,144,143,Y,1,Vince Velasquez,Huascar Ynoa,Velasquez hasn't pitched since his July 26 sea...,Ynoa will make his first big league start and ...,5,2,7.0,Bottom,2681,Citizens Bank Park,['TBS (out-of-market only)'],Series tied 1-1,Atlanta Braves,Philadelphia Phillies,Tyler Matzek,Deolis Guerra,Mark Melancon,2020-08-09 - Atlanta Braves (5) @ Philadelphia...,NaN


In [58]:

# Initialize a dictionary to hold our DataFrames
stats_matrices = {}

# Pitch Counts (already created)
#stats_matrices['Pitch Counts'] = pitcher_inning_matrix

# Strikeouts per Inning
strikeouts_per_inning = play_by_play[play_by_play['stat_category'] == 'Strikeout'].groupby(['pitcher_name', 'inning']).size().reset_index(name='strikeouts')
strikeouts_matrix = strikeouts_per_inning.pivot(index='pitcher_name', columns='inning', values='strikeouts').fillna(0).astype(int)
stats_matrices['Strikeouts'] = strikeouts_matrix

# Hits Allowed per Inning
hits_allowed_per_inning = play_by_play[play_by_play['stat_category'] == 'Hit'].groupby(['pitcher_name', 'inning']).size().reset_index(name='hits_allowed')
hits_matrix = hits_allowed_per_inning.pivot(index='pitcher_name', columns='inning', values='hits_allowed').fillna(0).astype(int)
stats_matrices['Hits Allowed'] = hits_matrix


# Add a 'Total' column to each DataFrame in the dictionary
for stat, df in stats_matrices.items():
    df['Total'] = df.sum(axis=1)



# Define all possible innings based on the play_by_play DataFrame to ensure we cover all innings in the game
all_innings_sorted = sorted(play_by_play['inning'].unique())

def ensure_all_innings_and_sort(df, all_innings_sorted):
    # Add missing inning columns with zeros
    for inning in all_innings_sorted:
        if inning not in df.columns:
            df[inning] = 0
    # Ensure the innings are in the correct order and append the 'Total' column last
    ordered_columns = sorted(df.columns, key=lambda x: x if isinstance(x, int) else float('inf'))
    if 'Total' in ordered_columns:
        ordered_columns.remove('Total')
    ordered_columns += ['Total']  # Make sure 'Total' is always the last column
    return df[ordered_columns]

# This function can be applied to any of the statistical DataFrames like so:
stats_matrices['Strikeouts'] = ensure_all_innings_and_sort(stats_matrices['Strikeouts'], all_innings_sorted)
stats_matrices['Hits Allowed'] = ensure_all_innings_and_sort(stats_matrices['Hits Allowed'], all_innings_sorted)

stats_matrices['Strikeouts']


inning,1,2,3,4,5,6,7,8,9,Total
pitcher_name,,,,,,,,,,
Blake Treinen,0,0,0,0,0,0,3,0,0,3
Blake Wood,0,0,0,6,0,0,0,0,0,6
Cam Bedrosian,0,0,0,0,5,0,0,0,0,5
Daniel Gossett,5,3,7,0,0,0,0,0,0,15
Jim Johnson,0,0,0,0,0,0,0,4,0,4
Jose Alvarez,0,0,0,0,0,8,0,0,0,8
Noé Ramirez,0,3,0,0,0,0,0,0,0,3
Santiago Casilla,0,0,0,0,0,0,0,12,0,12
Yusmeiro Petit,0,0,0,4,0,0,0,0,0,4


In [117]:
team_df = []


team_url = "https://sports.core.api.espn.com/v2/sports/baseball/leagues/mlb/teams"
team_data = requests.get(team_url).json()['items']

for key in team_data:
    team_url = key['$ref']
    team_info = requests.get(team_url).json()
    team_df.append({
        'id' : team_info['id'],
        'Team' : team_info['displayName']
    })

team_df = pd.DataFrame(team_df)

In [356]:
roster_df = []
record_df = []
next_game_df = []

game_id_list = []


for team_id in team_df['id'].to_list():


    team_url = f'https://site.api.espn.com/apis/site/v2/sports/baseball/mlb/teams/{team_id}?enable=roster,projection,stats'
    team_response = requests.get(team_url)
    team_data = team_response.json()

    team_name = team_data['team']['displayName']
    team_abbr = team_data['team']['abbreviation']
    team_id = team_data['team']['id']

    for athlete in team_data['team']['athletes']:
        roster_df.append(
            {
                'Team' : team_name,
                'Abbreviation' : team_abbr,
                'espn_team_id' : team_id,
                'espn_player_id' : athlete.get('id'),
                'Name' : athlete.get('displayName'),
                'weight' : athlete.get('weight'),
                'height' : athlete.get('height'),
                'age' : athlete.get('age'),
                'rookieYear' : athlete.get('debutYear'),
                'jersey' : athlete.get('jersey'),
                'position' : athlete.get('position', {}).get('abbreviation'),
                'experience' : athlete.get('experience', {}).get('years'),
                'isActive' : athlete.get('active'),
                'Bats' : athlete.get('bats', {}).get('abbreviation'),
                'Throws' : athlete.get('throws', {}).get('abbreviation'),
                'Injuries' : athlete.get('injuries')
            }
        )
    
    record_data = team_data['team']['record']
    team_dict = {}
    team_dict['Team'] = team_name
    team_dict['Abbreviation'] = team_abbr
    team_dict['espn_team_id'] = team_id
    for record_type in record_data['items']:
        game_type = record_type['type']
        for stat in record_type['stats']:
            team_dict[f"{game_type}_{stat['name']}"] = stat['value'] 
    team_dict['standingSummary'] = team_data['team']['standingSummary']
    record_df.append(team_dict)

    next_event = team_data['team']['nextEvent'][0]

    next_event_dict = {
            'Team' : team_name,
            'Abbreviation' :  team_abbr,
            'espn_team_id' : team_id,
            'espn_game_id' : next_event.get('id')
        }
    game_data = next_event['competitions'][0]

    next_event_dict['isNeutralSite'] = game_data['neutralSite']
    next_event_dict['game_time'] = game_data['status']['type']['shortDetail']
    next_event_dict['venue'] = game_data['venue']['fullName']

    for team in game_data['competitors']:
        
        if team['team']['displayName'] == team_name:
            next_event_dict['home_or_away'] = team['homeAway']
            next_event_dict['team_probable_starting_pitcher'] = team.get('probables',[{}])[0].get('athlete',{}).get('displayName')
            next_event_dict['team_probable_starting_pitcher_id'] = team.get('probables', [{}])[0].get('athlete',{}).get('id')
        else:
            next_event_dict['opponent'] = team['team']['displayName']
            next_event_dict['opponent_espn_team_id'] = team['team']['id']
            next_event_dict['opponent_probable_starting_pitcher'] = team.get('probables',[{}])[0].get('athlete',{}).get('displayName')
            next_event_dict['opponent_probable_starting_pitcher_id'] = team.get('probables', [{}])[0].get('athlete',{}).get('id')
    next_game_df.append(next_event_dict)




    




roster_df = pd.DataFrame(roster_df)
record_df = pd.DataFrame(record_df)
next_game_df = pd.DataFrame(next_game_df)

In [359]:
roster_df['Injuries'].to_list()

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [{'id': '-105707',
   'longComment': "The first part of the operation cleaned up scar tissue in the elbow, while the second moved the nerve to free it up from compression. While not routine, the follow-up procedures are sometimes needed for pitchers recovering from Tommy John surgery. Bautista's recovery is going well and the Orioles do not expect his readiness for the 2025 campaign to be affected at all.",
   'shortComment': 'Bautista underwent a right elbow debridement and an ulnar nerve transposition Friday, Jake Rill of MLB.com reports.',
   'status': '60-Day-IL',
   'date': '2024-02-14T18:02Z',
   'athlete': {},
   'source': {'id': '1', 'description': 'basic/manual', 'state': 'basic'},
   'type': {'id': '11',
    'name': 'INJURY_STATUS_60DAYIL',
    'description': '60-day IL',
    'abbreviation': 'IL60'},
   'details': {'fantasyStatus': {'description': '60-Day IL',
     'abbreviation': '60-Day IL'},
    'type': 'Elbow',
    '